# Titanic

In [16]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [17]:
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
          len(self.X), self.X.shape, self.y.shape
        )
        return str

### Trainning Data Set
해당 클래스는 학습을 위한 Dataset 클래스입니다.

<맵버 변수>
- X : Input Data Tensor
- y : Target Data Tensor

<맵버 함수>
- len() : Input Data의 길이 출력
- getitem(idx) : idx에 해당하는 Input, Target 값을 사전형으로 리턴
- str() : 해당 Data Set의 정보 출력

In [18]:
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(
          len(self.X), self.X.shape
        )
        return str

### Test Data Set
해당 클래스는 테스트를 위한 Dataset 클래스입니다.

<맵버 변수>
- X : Input Data Tensor

<맵버 함수>
- len() : Input Data의 길이 출력
- getitem(idx) : idx에 해당하는 Input 값을 사전형으로 리턴
- str() : 해당 Data Set의 정보 출력

In [19]:
def get_preprocessed_dataset():
    train_data_path = os.path.join(".", "Data", "train.csv")
    test_data_path = os.path.join(".", "Data", "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

### 데이터 생성 함수
- train_data_path : 학습 데이터의 파일 경로
- test_data_path : 테스트 데이터의 파일 경로
- train_df : 가공되지 않은 순수 학습 데이터
    - [418 rows x 12 columns]의 2차원 DataFrame
    - rows : Data의 수
    - columns : PassengerId(승객 번호), Survived(생존여부), Pclass(티켓 클래스), Name(이름), Sex(성별), Age(나이), SibSp(동승한 자매/배우자의 수), Parch(동승한 부모/자식의 수), Ticket(티켓 번호), Fare(승객 요금), Cabin(방 호수), Embarked(탑승지)
- test_df : 가공되지 않은 순수 테스트 데이터, train_df와 동일(row의 수만 다름)
- all_df : 데이터 가공을 위해 train_df, test_df를 병합한 데이터, 추가 함수를 이용하여 데이터를 가공
- train_x : 가공된 데이터에 대하여 "survived"가 결측값이 아니라면(train_data) 해당 행을 삭제합니다. 즉 학습 데이터의 데이터 값만을 가집니다.
- train_y : 학습데이터의 "survived"의 값만을 가져옵니다. 즉 target값 만을 가집니다.
- dataset : 가공된 데이터를 통하여 만들어진 학습에 사용될 데이터
- train_dataset, validation_dataset : 학습에 사용될 데이터를 8 : 2로 랜덤 분할한 학습 데이터, 검증 데이터
- test_dataset : 가공된 데이터를 통해 만들어진 테스트 데이터

In [20]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

### 1차 데이터 가공
- Fare 데이터의 결측치를 Pclass의 평균으로 대체

In [21]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

### 2차 데이터 가공
- 이름에 대하여 family_name(성), honorific(경칭), name(이름)으로 구분 후 총 데이터에 해당 3 컬럼으로 변경

In [22]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

### 3차 데이터 가공
- Age 데이터의 결측치를 honorific의 평균으로 대체

In [23]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

### 4차 데이터 가공
- SibSp(동승한 자매/배우자의 수), Parch(동승한 부모/자식의 수)의 데이터를 통하여 가족 수를 총 데이터에 추가
- 혼자 탑승자에 대한 컬럼을 따로 추가
- 학습에 불필요한 데이터 즉 생존에 연관 없는 데이터에 대한 삭제(승객 id, 이름, 성, 티켓 가격, 방 호수)

In [24]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

### 5차 데이터 가공
- 경칭에 대하여 Mr, Miss, Mrs, Master를 제외하고는 other로 묶기
- 탑승지가 결측치인 경우 missing의 값으로 대체

In [25]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

### 6차 데이터 가공
- category_features : all_df에서 데이터 타입이 "object"인 열들의 이름이 담김 즉, 이는 카테고리 변수들의 이름을 모아놓은 것
- LabelEncoder :  각 카테고리를 고유한 정수값으로 매핑
- LabelEncoder를 사용하여 카테고리 변수들을 수치값으로 변경 
- le = le.fit(all_df[category_feature])를 통해 해당 카테고리 변수의 고유한 값들을 학습
- all_df[category_feature] = le.transform(all_df[category_feature])를 통해 해당 카테고리 변수를 수치값으로 변환

# Titanic Trainning

In [60]:
import wandb
from torch import nn
from datetime import datetime

In [61]:
wandb.login()

True

In [62]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_input, 30),
            nn.RReLU(),
            nn.Linear(30, 50),
            nn.RReLU(),
            nn.Linear(50, 100),
            nn.RReLU(),
            nn.Linear(100, 50),
            nn.RReLU(),
            nn.Linear(50, 30),
            nn.RReLU(),
            nn.Linear(30, n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

### Model
- 입력 layer : intput by 30
- 히든 layer1 : 30 by 50
- 히든 layer2 : 50 by 100
- 히든 layer3 : 100 by 50
- 히든 layer4 : 50 by 30
- 출력 layer : 30 by output

- 활성화 함수
    - 입력 -> 히든1(ReLu)
    - 히든1 -> 히든2(sigmoid)
    - 히든2 -> 히든3(sigmoid)
    - 히든3 -> 히든4(sigmoid)
    - 히든4 -> 출력(ReLu)

In [63]:
def test(test_data_loader):
    print("[TEST]")
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    my_model = MyModel(n_input=11, n_output=2)
    output_batch = my_model(batch['input'])
    prediction_batch = torch.argmax(output_batch, dim=1)
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

### test
- batch : test_dasta_loader.shape -> torch.Size([418, 11])
- my_model : 입력 : 11, 출력 : 2
- output_batch : output_batch.shape -> ([418, 2])
- prediction_batch : output_batch.shape -> (418)

In [64]:
if __name__ == "__main__":
    ENV_NAME = "Kaggle_Titanic_FCN"
    use_wandb = True
    current_time = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
    config = {
        "env_name": ENV_NAME,
        "max_num_epoch": 100,
        "batch_size": 16,
        "learning_rate": 0.001,
    }
    
    if use_wandb:
            wandb = wandb.init(
                project=config["env_name"],
                name=current_time,
                config=config
            )
    
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))
        
    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    test(test_data_loader)
        
    my_model = MyModel(n_input=11, n_output=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(my_model.parameters(), lr=config["learning_rate"])

    for epoch in range(config["max_num_epoch"]):
        my_model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        for idx, batch in enumerate(train_data_loader):
            optimizer.zero_grad()
            input_data = batch['input']
            target = batch['target']

            output = my_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(train_data_loader)

        if use_wandb:
            wandb.log({"train_loss": average_loss, "train_accuracy": accuracy})

        print(f"Epoch [{epoch + 1}/{config['max_num_epoch']}], "
            f"Loss: {average_loss:.4f}, "
            f"Accuracy: {accuracy:.4f}")
        
        my_model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for idx, batch in enumerate(validation_data_loader):
                input_data = batch['input']
                target = batch['target']

                output = my_model(input_data)
                loss = criterion(output, target)

                total_loss += loss.item()

                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(validation_data_loader)

        if use_wandb:
            wandb.log({"val_loss": average_loss, "val_accuracy": accuracy})

        print(f"Validation Loss: {average_loss:.4f}, "
            f"Validation Accuracy: {accuracy:.4f}")
    wandb.finish()

        
    final_model_path = './Model/my_model_final.pth'
    torch.save(my_model.state_dict(), final_model_path)

    my_model = MyModel(n_input=11, n_output=2)
    my_model.load_state_dict(torch.load('./Model/my_model_final.pth'))
    my_model.eval()

    # 테스트 데이터를 DataLoader에 넣습니다.
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.
    predictions = []
    for batch in test_data_loader:
        inputs = batch['input']
        outputs = my_model(inputs)
        predictions_batch = torch.argmax(outputs, dim=1)
        predictions.extend(predictions_batch.tolist())

    # submission.csv 파일을 생성합니다.
    submission_df = pd.DataFrame({
        'PassengerId': range(892, 892 + len(predictions)),
        'Survived': predictions
    })

    # CSV 파일로 저장합니다.
    submission_df.to_csv('./Out/submission.csv', index=False)

    print("Submission 파일 'submission.csv'가 생성되었습니다.")

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  1.0000, 30.0000,  0.0000,  0.0000, 13.0000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
1 - tensor([ 2.0000,  0.0000, 22.0000,  1.0000,  1.0000, 29.0000,  2.0000, 21.1792,
         3.0000,  2.0000,  0.0000]): 1
2 - tensor([ 3.0000,  1.0000,  3.0000,  4.0000,  2.0000, 31.3875,  2.0000, 13.3029,
         0.0000,  6.0000,  0.0000]): 1
3 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  6.9500,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
4 - tensor([ 3.0000,  1.0000, 22.0000,  0.0000,  0.0000,  7.2500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 28.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 3.0000,  1.0000, 26.0000,  1.0000,  0.0000,  7.8542,  2.0000, 13.3029,
         2.0000,  1.0000,  0.0000]): 0
7 - tensor([ 1.00

Epoch [2/100], Loss: 0.6029, Accuracy: 0.6971
Validation Loss: 0.5719, Validation Accuracy: 0.7247
Epoch [3/100], Loss: 0.5995, Accuracy: 0.7223
Validation Loss: 0.5510, Validation Accuracy: 0.7416
Epoch [4/100], Loss: 0.5906, Accuracy: 0.7083
Validation Loss: 0.5716, Validation Accuracy: 0.7416
Epoch [5/100], Loss: 0.5726, Accuracy: 0.7251
Validation Loss: 0.5391, Validation Accuracy: 0.7079
Epoch [6/100], Loss: 0.5636, Accuracy: 0.7139
Validation Loss: 0.5639, Validation Accuracy: 0.7247
Epoch [7/100], Loss: 0.5753, Accuracy: 0.7195
Validation Loss: 0.5231, Validation Accuracy: 0.7247
Epoch [8/100], Loss: 0.5395, Accuracy: 0.7349
Validation Loss: 0.5515, Validation Accuracy: 0.7191
Epoch [9/100], Loss: 0.5320, Accuracy: 0.7321
Validation Loss: 0.5034, Validation Accuracy: 0.7697
Epoch [10/100], Loss: 0.5329, Accuracy: 0.7461
Validation Loss: 0.5622, Validation Accuracy: 0.7753
Epoch [11/100], Loss: 0.5138, Accuracy: 0.7630
Validation Loss: 0.4888, Validation Accuracy: 0.7640
Epoch [1

Epoch [87/100], Loss: 0.3442, Accuracy: 0.8668
Validation Loss: 0.5130, Validation Accuracy: 0.7472
Epoch [88/100], Loss: 0.3371, Accuracy: 0.8541
Validation Loss: 0.4924, Validation Accuracy: 0.7416
Epoch [89/100], Loss: 0.3417, Accuracy: 0.8527
Validation Loss: 0.5419, Validation Accuracy: 0.7584
Epoch [90/100], Loss: 0.3589, Accuracy: 0.8527
Validation Loss: 0.4828, Validation Accuracy: 0.7528
Epoch [91/100], Loss: 0.3336, Accuracy: 0.8555
Validation Loss: 0.5542, Validation Accuracy: 0.7416
Epoch [92/100], Loss: 0.3514, Accuracy: 0.8541
Validation Loss: 0.5481, Validation Accuracy: 0.7472
Epoch [93/100], Loss: 0.3474, Accuracy: 0.8513
Validation Loss: 0.4747, Validation Accuracy: 0.7528
Epoch [94/100], Loss: 0.3366, Accuracy: 0.8541
Validation Loss: 0.9133, Validation Accuracy: 0.7528
Epoch [95/100], Loss: 0.3291, Accuracy: 0.8668
Validation Loss: 0.5119, Validation Accuracy: 0.7640
Epoch [96/100], Loss: 0.3311, Accuracy: 0.8626
Validation Loss: 0.5813, Validation Accuracy: 0.7528


train_accuracy,▁▃▃▄▅▅▆▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇███▇▇██████
train_loss,█▇▆▆▅▅▄▅▄▃▃▃▃▃▂▂▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▁▁▁▁▁
val_accuracy,▃▄▃▃▆▇▅█▇▆▆▆▅▆▅▆▅▃▆▆▆▆▆▁▇▆▅▆▅█▅▄▄▆▅▆▅▅▆▅
val_loss,▂▂▃▂▁▂▁▁▁▁▂▂▃▁▁▂▂▂▂▁▃▁▂▄▂▂▃▄▂▂▂▅▁▂▂▂▂█▆▂
train_accuracy,0.85554
train_loss,0.34856
val_accuracy,0.74719
val_loss,0.53145


Submission 파일 'submission.csv'가 생성되었습니다.


![img](./Img/kaggle_titanic_res_2023-10-10.png)

# 후기

1. 활성함수 : 해당 contest를 진행하며 ReLU, Tanh, Sigmoid, ReLU6, ELU, LeakyReLU, RReLU를 섞어 사용해 보았습니다.
    - ReLu -> train_accuracy : 86.255%, val_accuracy : 77.528%
    - Tanh -> train_accuracy : 84.572%, val_accuracy : 82.022%
    - Sigmoid -> train_accuracy : 84.853%, val_accuracy : 84.27%
    - ReLU6 -> train_accuracy : 84.712%, val_accuracy : 82.022%
    - ELU -> train_accuracy : 86.957%, val_accuracy : 79.213%
    - LeakyReLU -> train_accuracy : 77.279%, val_accuracy : 81.461%
    - RReLU -> train_accuracy : 85.554%, val_accuracy : 74.719%
    위와 같은 성능을 보여줬으며 위의 결과는 단일 시행에 대한 결과이며 평균적인 성능의 지표는 아닙니다.
    단일 시행 시 ELU함수가 학습 데이터에 대하여 가작 높은 정확도를 보였으며 검증 데이터에 대하여는 ReLU6가 가장 높은 정확도를 보였습니다.

2. 검증 : 해당 코드에서는 매 epoch마다 검증을 시행하고 있습니다.
    - 해당 코드의 경우 데이터의 크기가 작고 신경망의 깊이가 깊지 않아 매 epoch마다 검증을 시행했습니다. 
    - 만약 신경망의 깊이가 깊거나 배치학습을 돌리더라도 데이터가 매우 많은 경우는 학습에 시간이 오래 걸림으로 모니터링을 하는 것을 특정 epoch의 횟수를 완료할 때 마다 검증을 하는 것이 더 효율적인 것 같습니다.